In [1]:
from Model import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import yfinance as yf

In [2]:
tickers = ['AGG', 'DBC', 'VIXY', 'VTI']
start_date = '2006-02-06'
end_date = '2020-04-06'
trading_days = 252
np.random.seed(123)

In [ ]:
class Portfolio:
    def __init__(self, initial_cash: float, assets: pd.DataFrame):
        """
        Initializes the Portfolio object.

        Parameters:
        - initial_cash: The starting value of the portfolio in cash.
        - assets: DataFrame of asset prices (historical data).
        """
        self.initial_cash = initial_cash
        self.current_value = initial_cash
        self.assets = assets  # Historical price data for the assets
        self.weights = np.zeros(len(assets.columns))  # Initialize weights as zero
        self.portfolio_history = []  # To track portfolio value over time
    
    def set_weights(self, weights: np.array):
        """
        Set the weights of the assets in the portfolio.

        Parameters:
        - weights: Array of weights corresponding to the assets.
        """
        self.weights = weights

    def num_shares(self, prices: np.array):
        """
        Calculate the number of shares of each asset in the portfolio.

        Parameters:
        - prices: Array of current prices of the assets.

        Returns:
        - Array of number of shares of each asset.
        """
        return self.current_value * self.weights / prices
    
    
